# Libraries and Global Variables

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
from IPython.display import display, Markdown

#import arcgis libraries
from arcgis.gis import *
gis = GIS()

In [16]:
working_directory = os.getcwd()
data_folder         = os.path.join(working_directory, "data"        )
intermediate_folder = os.path.join(working_directory, "intermediate")
results_folder      = os.path.join(working_directory, "results"     )
acs_folder          = os.path.join(data_folder      , "ACS"         )
taz_folder          = os.path.join(working_directory, "TAZ"         )
acs_filename        = "ACSST5Y2019.S1401_data_with_overlays_2022-01-19T171953.csv"

shp_taz =  os.path.join(taz_folder, r"USTM_TAZ_2021_09_22.shp")

shp_huitaz = os.path.join(intermediate_folder, r"wfrc_hui_centroid_withTAZ.shp")

#USED FOR UTAH COUNTY SINCE 2019 NOT AVAILABLE YET. NEED TO UPDATE WITH 2019 BASE YEAR DATA WHEN AVAILABLE.
remm_folder          = os.path.join(data_folder,"REMM")
parcel_filename      = os.path.join(remm_folder, r"parcels_bldg_20210623\parcels\Center01102019.gdb\BaseGP01102019")
parcel_info_filename = os.path.join(remm_folder, r"parcels_bldg_20210623\BASE_run_285_2015_parcel_dev.csv"         )
#taz_filename = os.path.join(taz_folder,"TAZ.shp")

#Housing Inventory
wfrc_hui = os.path.join(remm_folder, r"wfrc_hui_20220106.gdb\wfrc_housing_unit_inventory_20220106")
wfrc_hui_centroids = os.path.join(intermediate_folder, r"wfrc_hui_centroid.shp")

#column codes from ACS data, will verify later in process
public_col = "S1401_C03_010E" #Estimate!!In public school!!Population enrolled in college or graduate school
private_col = "S1401_C05_010E" #Estimate!!In private school!!Population enrolled in college or graduate school
total_col = "S1401_C01_010E" #Estimate!!Total!!Population enrolled in college or graduate school

public = "ACS_Public"
private = "ACS_Private"
total = "ACS_Total"

print(working_directory)
print(data_folder)
print(results_folder)
print(acs_folder)

def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)

E:\GitHub\TDM-College-Enrollment-v9
E:\GitHub\TDM-College-Enrollment-v9\data
E:\GitHub\TDM-College-Enrollment-v9\results
E:\GitHub\TDM-College-Enrollment-v9\data\ACS


# Gather MF and SF Units by TAZ

## MAG Area

In [3]:
#Read in REMM Parcel
sdf_parcel = pd.DataFrame.spatial.from_featureclass(parcel_filename)

#only use for Utah County
sdf_parcel = sdf_parcel[sdf_parcel['county_name']=='UTAH']

#only need a few columns
sdf_parcel = sdf_parcel[['parcel_id','parcel_id_REMM','x','y','SHAPE']]
display (sdf_parcel)

,parcel_id,parcel_id_REMM,x,y,SHAPE
0,671124.0,671124.0,1.572705e+06,7.294321e+06,"{""rings"": [[[1572658.531162396, 7294412.922931..."
1,588354.0,588354.0,1.572481e+06,7.293816e+06,"{""rings"": [[[1572446.1962272525, 7294150.73555..."
2,664267.0,664267.0,1.571343e+06,7.291668e+06,"{""rings"": [[[1571352.9701225907, 7291780.92202..."
3,632453.0,632453.0,1.580054e+06,7.303541e+06,"{""rings"": [[[1579974.9191654772, 7303604.60588..."
4,632450.0,632450.0,1.580190e+06,7.303272e+06,"{""rings"": [[[1580242.158429563, 7303338.493389..."
...,...,...,...,...,...
845214,601655.0,886695.0,1.574259e+06,7.220958e+06,"{""rings"": [[[1574478.0519851, 7220743.71421465..."
845215,601655.0,886696.0,1.573685e+06,7.220955e+06,"{""rings"": [[[1573850.763353318, 7220743.714214..."
845216,601655.0,886697.0,1.573302e+06,7.220958e+06,"{""rings"": [[[1573537.6948660016, 7220743.71421..."
845217,601655.0,886698.0,1.572836e+06,7.220958e+06,"{""rings"": [[[1573072.201039672, 7220743.714214..."


In [4]:
#check attribute of bounding box of zip sdf
#print(sdf_parcel.iloc[0].SHAPE)

#create sdf of parcel centroids, using projection 2281: NAD83 / Utah Central (ft)
sdf_parcel_centroid =  pd.DataFrame.spatial.from_xy(sdf_parcel, 'x', 'y', 2281)
sdf_parcel_centroid

#print(sdf_parcel_centroid.iloc[0].SHAPE)

#csv has two header rows
sdf_taz = pd.DataFrame.spatial.from_featureclass(shp_taz)
display(sdf_taz.head())

#print(sdf_taz.iloc[0].SHAPE)

#change projection to NAD83 / Utah Central (ft) to match tracts
sdf_taz.spatial.project(2281)
#print(sdf_taz.iloc[0].SHAPE)

#spatial join parcel centroid data to taz data
sdf_parcel_taz_join = sdf_parcel_centroid.spatial.join(sdf_taz)

display(sdf_parcel_taz_join.head())

,FID,TAZID,SA_TAZID,CO_IDX,CO_TAZID,SUBAREAID,ACRES,DEVACRES,DEVPBLEPCT,X,...,SORT,TAZID_V21B,SATAZ_V21B,COTAZ_V21B,SUBID_V21B,TAZID_V21A,SATAZ_V21A,COTAZ_V21A,SUBID_V21A,SHAPE
0,0,43,3,3,1003,0,440.680556,440.680556,1.00,357308.618265,...,1,43,0,1013,0,43,0,1013,0,"{""rings"": [[[357893.0658, 4241258.5964], [3578..."
1,1,45,5,5,1005,0,245.424579,245.424579,1.00,356023.333451,...,2,45,0,1015,0,45,0,1015,0,"{""rings"": [[[356154.68740000017, 4239646.444],..."
2,2,46,6,6,1006,0,675.848293,675.848293,1.00,357530.306637,...,3,46,0,1016,0,46,0,1016,0,"{""rings"": [[[356651.19880000036, 4239664.66310..."
3,3,55,15,15,1015,0,550.767119,534.244106,0.97,356277.254346,...,4,55,0,1025,0,55,0,1025,0,"{""rings"": [[[356587.71449999977, 4236519.61900..."
4,4,50,10,10,1010,0,176.207617,176.207617,1.00,355998.119155,...,5,50,0,1020,0,50,0,1020,0,"{""rings"": [[[356438.3099999996, 4237510.316600..."


,parcel_id,parcel_id_REMM,x,y,SHAPE,index_right,FID,TAZID,SA_TAZID,CO_IDX,...,DSML_NAME,SORT,TAZID_V21B,SATAZ_V21B,COTAZ_V21B,SUBID_V21B,TAZID_V21A,SATAZ_V21A,COTAZ_V21A,SUBID_V21A
0,671124.0,671124.0,1.572705e+06,7.294321e+06,"{""spatialReference"": {""wkid"": 2281}, ""x"": 1572...",6132,6132,6318,2773,557,...,Lindon,6133,5525,2246,492246,1,5801,2246,492246,1
1,588354.0,588354.0,1.572481e+06,7.293816e+06,"{""spatialReference"": {""wkid"": 2281}, ""x"": 1572...",6132,6132,6318,2773,557,...,Lindon,6133,5525,2246,492246,1,5801,2246,492246,1
2,664267.0,664267.0,1.571343e+06,7.291668e+06,"{""spatialReference"": {""wkid"": 2281}, ""x"": 1571...",6132,6132,6318,2773,557,...,Lindon,6133,5525,2246,492246,1,5801,2246,492246,1
3,670759.0,670759.0,1.572231e+06,7.295041e+06,"{""spatialReference"": {""wkid"": 2281}, ""x"": 1572...",6132,6132,6318,2773,557,...,Lindon,6133,5525,2246,492246,1,5801,2246,492246,1
4,648127.0,648127.0,1.573825e+06,7.289964e+06,"{""spatialReference"": {""wkid"": 2281}, ""x"": 1573...",6132,6132,6318,2773,557,...,Lindon,6133,5525,2246,492246,1,5801,2246,492246,1


In [5]:
#create index table of parcel id and TAZ ID
df_parcel_taz = sdf_parcel_taz_join[['parcel_id','CO_TAZID']]
display(df_parcel_taz)

,parcel_id,CO_TAZID
0,671124.0,490557
1,588354.0,490557
2,664267.0,490557
3,670759.0,490557
4,648127.0,490557
...,...,...
250205,739015.0,490100
250206,739015.0,490100
250207,739015.0,490100
250208,739015.0,490100


In [6]:
#Read in REMM Building
df_parcel_resunitinfo = pd.read_csv(parcel_info_filename, usecols=['parcel_id','is_sf','is_mf','residential_units'])

#only include residential general_type
df_parcel_resunitinfo = df_parcel_resunitinfo[(df_parcel_resunitinfo.is_sf > 0) | (df_parcel_resunitinfo.is_mf > 0)]
display(df_parcel_resunitinfo)

,parcel_id,residential_units,is_sf,is_mf
0,741871,1,1.0,0.0
3,640185,1,1.0,0.0
4,682698,1,1.0,0.0
6,693553,1,1.0,0.0
7,658088,1,1.0,0.0
...,...,...,...,...
841528,601446,1,1.0,0.0
841575,609702,1,1.0,0.0
841587,603550,1,1.0,0.0
841621,581782,1,1.0,0.0


In [7]:
#join
df_parcel_resunitinfo_taz = pd.merge(df_parcel_resunitinfo, df_parcel_taz, on="parcel_id", how="inner")

#create TYPE and UNIT_COUNT columns to match WFRC housing inventory data
df_parcel_resunitinfo_taz.loc[(df_parcel_resunitinfo_taz['is_sf'] > 0), 'TYPE'] = 'SF_Units'
df_parcel_resunitinfo_taz.loc[(df_parcel_resunitinfo_taz['is_mf'] > 0), 'TYPE'] = 'MF_Units'
df_parcel_resunitinfo_taz = df_parcel_resunitinfo_taz[['CO_TAZID','parcel_id','TYPE','residential_units']]
df_parcel_resunitinfo_taz = df_parcel_resunitinfo_taz.rename(columns={"residential_units": "UNIT_COUNT"})
display(df_parcel_resunitinfo_taz)

display(df_parcel_resunitinfo_taz.groupby(['TYPE'],as_index=False).agg(Units=('UNIT_COUNT','sum')))

df_TAZ_resunitcount_MAG = df_parcel_resunitinfo_taz.groupby(['CO_TAZID','TYPE'],as_index=False).agg(UNIT_COUNT=('UNIT_COUNT','sum'))
display(df_TAZ_resunitcount_MAG)
display(df_TAZ_resunitcount_MAG['UNIT_COUNT'].sum())

,CO_TAZID,parcel_id,TYPE,UNIT_COUNT
0,490420,741871,SF_Units,1
1,490402,640185,SF_Units,1
2,490435,682698,SF_Units,1
3,490463,693553,SF_Units,1
4,490465,658088,SF_Units,1
...,...,...,...,...
128014,491037,601655,SF_Units,1
128015,491037,601655,SF_Units,1
128016,491037,601655,SF_Units,1
128017,491037,601655,SF_Units,1


,TYPE,Units
0,MF_Units,48548
1,SF_Units,118006


,CO_TAZID,TYPE,UNIT_COUNT
0,351251,SF_Units,2
1,490003,SF_Units,1
2,490004,SF_Units,3
3,490005,MF_Units,1
4,490005,SF_Units,3
...,...,...,...
1319,491279,SF_Units,1
1320,491280,MF_Units,6
1321,491280,SF_Units,15
1322,491307,SF_Units,6


166554

In [8]:
#clear MAG dataframes
try:
    del sdf_parcel
# catch when df1 is None
except AttributeError:
    pass
# catch when it hasn't even been defined
except NameError:
    pass

try:
    del sdf_parcel_centroid
# catch when df1 is None
except AttributeError:
    pass
# catch when it hasn't even been defined
except NameError:
    pass

try:
    del sdf_parcel_taz_join
# catch when df1 is None
except AttributeError:
    pass
# catch when it hasn't even been defined
except NameError:
    pass

try:
    del df_parcel_resunitinfo_taz
# catch when df1 is None
except AttributeError:
    pass
# catch when it hasn't even been defined
except NameError:
    pass


## WFRC Area

In [17]:
#Compute TAZ Unit Counts for WFRC area using Housing Inventory
import arcpy

deleteIfExists(wfrc_hui_centroids)

arcpy.management.FeatureToPoint(wfrc_hui, wfrc_hui_centroids, "INSIDE")

<Result 'E:\\GitHub\\TDM-College-Enrollment-v9\\intermediate\\wfrc_hui_centroid.shp'>

In [18]:
arcpy.DeleteField_management(wfrc_hui_centroids,["SUBTYPE","NOTE","IS_OUG","CITY","COUNTY","SUBCOUNTY","HOUSE_CNT","ADDR_CNT","DUA","APX_HGHT","ACRES","TOT_BD_FT2","TOT_VALUE","APX_BLT_YR","BLT_DECADE"])

#Read in HUI Centroids
sdf_HUI_Centroids = pd.DataFrame.spatial.from_featureclass(wfrc_hui_centroids)

In [19]:
sdf_HUI_Centroids.spatial.project(2281)
print(sdf_taz.iloc[0].SHAPE)

{'rings': [[[1317422.8107823327, 6554548.490476552], [1317322.56938774, 6549246.932812534], [1317318.266676752, 6549244.1204259265], [1317304.1594533115, 6549241.019081637], [1317265.3966676956, 6549240.778313604], [1317231.2364678606, 6549241.208144006], [1317103.4936737572, 6549242.816395033], [1316782.8539483796, 6549247.584622337], [1316441.555040642, 6549253.3462799415], [1316220.8225590726, 6549257.589504647], [1316182.6433918711, 6549258.071824613], [1316067.5406368854, 6549260.254139651], [1316029.0793361575, 6549261.104305551], [1315740.2992788283, 6549265.118766269], [1314873.9595586692, 6549277.181474307], [1314584.8973357358, 6549281.576655959], [1314343.7698100007, 6549285.002293314], [1314125.033339461, 6549288.145357976], [1313649.9782815452, 6549296.737068523], [1313619.827691839, 6549296.392132035], [1313378.371447676, 6549296.552259902], [1313373.0174651097, 6549296.627918396], [1313487.6257336494, 6553319.321004086], [1313491.931230892, 6553319.205336077], [1314745.7

In [20]:
#spatial join parcel centroid data to taz data
sdf_hui_taz_join = sdf_HUI_Centroids.spatial.join(sdf_taz)

In [21]:
sdf_hui_taz_join

,FID_left,Shape_Leng,Shape_Area,TYPE,UNIT_COUNT,UNIT_ID,ORIG_FID,SHAPE,index_right,FID_right,...,DSML_NAME,SORT,TAZID_V21B,SATAZ_V21B,COTAZ_V21B,SUBID_V21B,TAZID_V21A,SATAZ_V21A,COTAZ_V21A,SUBID_V21A
0,0,436.253371,7799.559612,single_family,18.0,1.0,1,"{""x"": 1528616.9518034789, ""y"": 7487223.2140268...",3946,3946,...,Woods Cross,3947,3910,624,110624,1,4186,624,110624,1
1,18527,131.573539,1036.891642,single_family,1.0,18528.0,18528,"{""x"": 1528865.6660156823, ""y"": 7487555.5086987...",3946,3946,...,Woods Cross,3947,3910,624,110624,1,4186,624,110624,1
2,18534,176.293080,1344.371842,single_family,1.0,18535.0,18535,"{""x"": 1528940.3036255226, ""y"": 7487507.7230442...",3946,3946,...,Woods Cross,3947,3910,624,110624,1,4186,624,110624,1
3,18556,197.998975,1881.269075,single_family,1.0,18557.0,18557,"{""x"": 1529106.8609069497, ""y"": 7486433.4920012...",3946,3946,...,Woods Cross,3947,3910,624,110624,1,4186,624,110624,1
4,18557,148.060750,1141.533879,single_family,1.0,18558.0,18558,"{""x"": 1529120.4073760984, ""y"": 7486537.6425659...",3946,3946,...,Woods Cross,3947,3910,624,110624,1,4186,624,110624,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391827,391669,347.018403,7475.986227,multi_family,96.0,391675.0,391675,"{""x"": 1530265.248649002, ""y"": 7446679.41962527...",4147,4147,...,Salt Lake City,4148,4249,963,350963,1,4525,963,350963,1
391828,391662,296.782462,5458.624019,multi_family,186.0,391668.0,391668,"{""x"": 1528751.3668371248, ""y"": 7450639.7370281...",4068,4068,...,Salt Lake City,4069,4321,1035,351035,1,4597,1035,351035,1
391829,391663,356.285514,7868.438907,multi_family,265.0,391669.0,391669,"{""x"": 1528481.5562136997, ""y"": 7450640.7171658...",4068,4068,...,Salt Lake City,4069,4321,1035,351035,1,4597,1035,351035,1
391830,391664,665.553734,23825.276568,multi_family,489.0,391670.0,391670,"{""x"": 1528642.2288564895, ""y"": 7451258.3385253...",4068,4068,...,Salt Lake City,4069,4321,1035,351035,1,4597,1035,351035,1


In [22]:
#export intermediate shapefile
sdf_hui_taz_join_copy = sdf_hui_taz_join[['FID_left','TYPE','UNIT_COUNT','CO_TAZID','SHAPE']].copy()
deleteIfExists(shp_huitaz)
sdf_hui_taz_join_copy.spatial.to_featureclass(shp_huitaz,sanitize_columns=False)
print(sdf_hui_taz_join_copy.iloc[0].SHAPE)
display(sdf_hui_taz_join_copy)

{'x': 1528616.9518034789, 'y': 7487223.214026821, 'spatialReference': {'wkid': 2281, 'latestWkid': 2281}}


,FID_left,TYPE,UNIT_COUNT,CO_TAZID,SHAPE
0,0,single_family,18.0,110289,"{""x"": 1528616.9518034789, ""y"": 7487223.2140268..."
1,18527,single_family,1.0,110289,"{""x"": 1528865.6660156823, ""y"": 7487555.5086987..."
2,18534,single_family,1.0,110289,"{""x"": 1528940.3036255226, ""y"": 7487507.7230442..."
3,18556,single_family,1.0,110289,"{""x"": 1529106.8609069497, ""y"": 7486433.4920012..."
4,18557,single_family,1.0,110289,"{""x"": 1529120.4073760984, ""y"": 7486537.6425659..."
...,...,...,...,...,...
391827,391669,multi_family,96.0,350166,"{""x"": 1530265.248649002, ""y"": 7446679.41962527..."
391828,391662,multi_family,186.0,350087,"{""x"": 1528751.3668371248, ""y"": 7450639.7370281..."
391829,391663,multi_family,265.0,350087,"{""x"": 1528481.5562136997, ""y"": 7450640.7171658..."
391830,391664,multi_family,489.0,350087,"{""x"": 1528642.2288564895, ""y"": 7451258.3385253..."


In [23]:
df_hui_taz = sdf_hui_taz_join[['CO_TAZID','TYPE','UNIT_COUNT',]]
df_hui_taz.loc[(df_hui_taz['TYPE']=='single_family'), 'TYPE'] = 'SF_Units'
df_hui_taz.loc[(df_hui_taz['TYPE']=='multi_family' ), 'TYPE'] = 'MF_Units'
df_hui_taz['UNIT_COUNT'] = df_hui_taz['UNIT_COUNT'].astype('int64')
df_hui_taz

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,CO_TAZID,TYPE,UNIT_COUNT
0,110289,SF_Units,18
1,110289,SF_Units,1
2,110289,SF_Units,1
3,110289,SF_Units,1
4,110289,SF_Units,1
...,...,...,...
391827,350166,MF_Units,96
391828,350087,MF_Units,186
391829,350087,MF_Units,265
391830,350087,MF_Units,489


In [24]:
#aggregate by COTAZID
df_TAZ_resunitcount_WFRC = df_hui_taz.groupby(['CO_TAZID','TYPE'],as_index=False).agg(UNIT_COUNT=('UNIT_COUNT','sum'))
df_TAZ_resunitcount_WFRC

,CO_TAZID,TYPE,UNIT_COUNT
0,110001,SF_Units,13
1,110002,SF_Units,56
2,110003,SF_Units,27
3,110004,MF_Units,4
4,110004,SF_Units,29
...,...,...,...
2776,570421,MF_Units,74
2777,570421,SF_Units,23
2778,570422,MF_Units,2
2779,570422,SF_Units,301


# Combine and Summarize

In [25]:
#combine WFRC and MAG
df_TAZ_resunitcount = pd.concat([df_TAZ_resunitcount_WFRC, df_TAZ_resunitcount_MAG])

df_TAZ_resunitcount

,CO_TAZID,TYPE,UNIT_COUNT
0,110001,SF_Units,13
1,110002,SF_Units,56
2,110003,SF_Units,27
3,110004,MF_Units,4
4,110004,SF_Units,29
...,...,...,...
1319,491279,SF_Units,1
1320,491280,MF_Units,6
1321,491280,SF_Units,15
1322,491307,SF_Units,6


In [26]:
df_TAZ_resunit = pd.pivot_table(df_TAZ_resunitcount, values='UNIT_COUNT', index=['CO_TAZID'],
                    columns=['TYPE'], aggfunc=np.sum)
df_TAZ_resunit = df_TAZ_resunit.fillna(0)
df_TAZ_resunit

TYPE,MF_Units,SF_Units
CO_TAZID,,
110001,0.0,13.0
110002,0.0,56.0
110003,0.0,27.0
110004,4.0,29.0
110005,0.0,26.0
...,...,...
570419,772.0,125.0
570420,0.0,293.0
570421,74.0,23.0


In [27]:
df_TAZ_resunit.to_csv(os.path.join(intermediate_folder,r"TAZResUnits.csv"))